In [4]:
# Install required packages
!pip install groq python-dotenv langchain-core langchain-groq

# Import necessary libraries
import os
from groq import Groq
from typing import Dict, List, Optional
import json
from datetime import datetime
import requests

# Set up Groq API (You'll need to get a free API key from https://console.groq.com/)
os.environ["GROQ_API_KEY"] = "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"  # Replace with your actual key

# Initialize Groq client
client = Groq(api_key=os.environ["GROQ_API_KEY"])

class SimpleAgent:
    def __init__(self, model: str = "llama-3.1-8b-instant"):
        self.client = Groq(api_key=os.environ["GROQ_API_KEY"])
        self.model = model
        self.conversation_history = []

    def add_to_history(self, role: str, content: str):
        """Add message to conversation history"""
        self.conversation_history.append({
            "role": role,
            "content": content,
            "timestamp": datetime.now().isoformat()
        })

    def get_available_models(self) -> List[str]:
        """Get list of CURRENTLY AVAILABLE Groq models"""
        return [
            "llama-3.1-8b-instant",  # This one works!
            "llama-3.2-1b-preview",  # New smaller model
            "llama-3.2-3b-preview",  # New medium model
            "llama-3.2-90b-vision-preview",  # Large model with vision
            "llama-3.2-11b-vision-preview",  # Medium with vision
            "llama-3.2-90b-text-preview",  # Large text model
            "mixtral-8x7b-32768"  # Some legacy might work
        ]

    def get_structured_response(self,
                              prompt: str,
                              response_format: Dict = None,
                              max_tokens: int = 1024,
                              temperature: float = 0.7) -> Dict:
        """
        Get structured response from the agent
        """

        # Default response format if none provided
        if response_format is None:
            response_format = {
                "type": "object",
                "properties": {
                    "summary": {"type": "string"},
                    "key_points": {"type": "array", "items": {"type": "string"}},
                    "action_items": {"type": "array", "items": {"type": "string"}},
                    "sentiment": {"type": "string", "enum": ["positive", "negative", "neutral"]},
                    "confidence_score": {"type": "number", "minimum": 0, "maximum": 1}
                },
                "required": ["summary", "key_points", "action_items", "sentiment", "confidence_score"]
            }

        # System prompt for structured responses
        system_prompt = f"""
        You are a helpful AI assistant that provides structured responses.
        Always respond in valid JSON format matching this schema:
        {json.dumps(response_format, indent=2)}

        Ensure your response is parseable JSON and follows the schema exactly.
        Return ONLY the JSON object, no additional text.
        """

        try:
            # Create chat completion
            completion = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                max_tokens=max_tokens,
                top_p=1,
                stream=False,
                response_format={"type": "json_object"}
            )

            response_content = completion.choices[0].message.content

            # Parse JSON response
            structured_response = json.loads(response_content)

            # Add to history
            self.add_to_history("user", prompt)
            self.add_to_history("assistant", json.dumps(structured_response, indent=2))

            return structured_response

        except Exception as e:
            error_msg = f"Failed to get response: {str(e)}"
            self.add_to_history("system", error_msg)
            return {"error": error_msg}

    def print_conversation_history(self):
        """Print the conversation history"""
        print("\n" + "="*50)
        print("CONVERSATION HISTORY")
        print("="*50)
        for i, message in enumerate(self.conversation_history):
            print(f"\n[{i+1}] {message['role'].upper()} ({message['timestamp']}):")
            if message['role'] == 'assistant':
                try:
                    # Pretty print JSON responses
                    parsed = json.loads(message['content'])
                    print(json.dumps(parsed, indent=2))
                except:
                    print(message['content'])
            else:
                print(message['content'])
        print("="*50)

# Test ALL current models
def test_all_models():
    """Test all potential Groq models"""
    agent = SimpleAgent()

    # Extended list of possible models
    all_possible_models = [
        "llama-3.1-8b-instant",
        "llama-3.2-1b-preview",
        "llama-3.2-3b-preview",
        "llama-3.2-90b-vision-preview",
        "llama-3.2-11b-vision-preview",
        "llama-3.2-90b-text-preview",
        "llama-3.2-1b-text-preview",
        "llama-3.2-3b-text-preview",
        "mixtral-8x7b-32768",
        "gemma2-9b-it",
        "llama3-8b-8192",
        "llama3-70b-8192"
    ]

    print("🧪 Testing ALL Potential Groq Models...")
    working_models = []

    test_prompt = "What are 2 benefits of solar energy? Return as JSON."

    for model in all_possible_models:
        print(f"\nTesting: {model}")
        try:
            agent.model = model
            response = agent.get_structured_response(test_prompt)

            if "error" not in response:
                print(f"✅ SUCCESS - {model}")
                working_models.append(model)
                # Show a snippet of the response
                summary = response.get('summary', 'N/A')[:100] + "..." if response.get('summary') else 'N/A'
                print(f"   Response: {summary}")
            else:
                print(f"❌ FAILED - {model}")
                # Show error message
                error_msg = response['error']
                if "decommissioned" in error_msg:
                    print("   Reason: Model decommissioned")
                elif "not found" in error_msg:
                    print("   Reason: Model not found")
                else:
                    print(f"   Reason: {error_msg[:100]}...")

        except Exception as e:
            print(f"❌ ERROR - {model}: {str(e)[:100]}...")

    print(f"\n🎯 WORKING MODELS: {working_models}")
    return working_models

# Example usage with guaranteed working model
def demo_structured_agent():
    # Use the confirmed working model
    agent = SimpleAgent(model="llama-3.1-8b-instant")

    print("🤖 Simple Groq Agent Demo")
    print(f"Using model: {agent.model}")

    # Example 1: Basic structured response
    print("\n" + "="*60)
    print("1. Basic Structured Response:")
    print("="*60)
    prompt1 = "Explain the benefits of renewable energy in 3 key points"
    response1 = agent.get_structured_response(prompt1)
    print("Prompt:", prompt1)
    print("Response:")
    print(json.dumps(response1, indent=2))

    # Example 2: Custom response format
    print("\n" + "="*60)
    print("2. Custom Structured Response:")
    print("="*60)
    custom_format = {
        "type": "object",
        "properties": {
            "product_name": {"type": "string"},
            "features": {"type": "array", "items": {"type": "string"}},
            "target_audience": {"type": "string"},
            "price_estimate": {"type": "number"},
            "development_timeline": {"type": "string"}
        },
        "required": ["product_name", "features", "target_audience", "price_estimate", "development_timeline"]
    }

    prompt2 = "Create a product idea for a smart home device for elderly people"
    response2 = agent.get_structured_response(prompt2, response_format=custom_format)
    print("Prompt:", prompt2)
    print("Response:")
    print(json.dumps(response2, indent=2))

    # Example 3: Analysis with structured output
    print("\n" + "="*60)
    print("3. Analysis with Structured Output:")
    print("="*60)
    analysis_format = {
        "type": "object",
        "properties": {
            "topic": {"type": "string"},
            "strengths": {"type": "array", "items": {"type": "string"}},
            "weaknesses": {"type": "array", "items": {"type": "string"}},
            "opportunities": {"type": "array", "items": {"type": "string"}},
            "threats": {"type": "array", "items": {"type": "string"}},
            "overall_assessment": {"type": "string"}
        },
        "required": ["topic", "strengths", "weaknesses", "opportunities", "threats", "overall_assessment"]
    }

    prompt3 = "Perform a SWOT analysis for electric vehicles in 2024"
    response3 = agent.get_structured_response(prompt3, response_format=analysis_format)
    print("Prompt:", prompt3)
    print("Response:")
    print(json.dumps(response3, indent=2))

    # Show conversation history
    agent.print_conversation_history()

# Advanced agent with tools
class AdvancedAgent(SimpleAgent):
    def __init__(self, model: str = "llama-3.1-8b-instant"):
        super().__init__(model)
        self.tools = {
            "calculator": self.calculate,
            "time": self.get_current_time,
            "text_analyzer": self.analyze_text
        }

    def calculate(self, expression: str) -> float:
        """Simple calculator tool"""
        try:
            # Safety check - only allow basic math operations
            allowed_chars = set('0123456789+-*/.() ')
            if all(c in allowed_chars for c in expression):
                return eval(expression)
            else:
                return "Error: Invalid characters in expression"
        except:
            return "Error: Could not evaluate expression"

    def get_current_time(self) -> str:
        """Get current time tool"""
        return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    def analyze_text(self, text: str) -> Dict:
        """Simple text analysis tool"""
        words = len(text.split())
        chars = len(text)
        return {
            "word_count": words,
            "character_count": chars,
            "reading_time_minutes": round(words / 200, 2)
        }

    def process_with_tools(self, prompt: str) -> Dict:
        """Process prompt with available tools"""

        tool_result = "No tool used"
        tool_used = "none"

        # Simple tool detection
        if any(word in prompt.lower() for word in ['calculate', 'math', '+', '-', '*', '/']):
            tool_used = "calculator"
            # Extract simple math expressions
            import re
            numbers = re.findall(r'\d+\.?\d*', prompt)
            if numbers and len(numbers) >= 2:
                tool_result = str(self.calculate('+'.join(numbers)))
            else:
                tool_result = "Could not extract calculation"

        elif "time" in prompt.lower():
            tool_used = "time"
            tool_result = self.get_current_time()

        elif "analyze" in prompt.lower() and "text" in prompt.lower():
            tool_used = "text_analyzer"
            tool_result = str(self.analyze_text(prompt))

        # Get final response
        tool_format = {
            "type": "object",
            "properties": {
                "analysis": {"type": "string"},
                "tool_used": {"type": "string"},
                "tool_result": {"type": "string"},
                "final_answer": {"type": "string"}
            },
            "required": ["analysis", "tool_used", "tool_result", "final_answer"]
        }

        final_prompt = f"""
        Original prompt: {prompt}
        Tool used: {tool_used}
        Tool result: {tool_result}

        Provide a comprehensive answer using the tool result if applicable.
        """

        return self.get_structured_response(final_prompt, response_format=tool_format)

# Run the complete demo
if __name__ == "__main__":
    print("🚀 Starting Groq Agent with Current Models...")

    # First, test all models to find working ones
    working_models = test_all_models()

    if working_models:
        print(f"\n🎉 Found {len(working_models)} working model(s): {working_models}")

        # Run main demo with first working model
        print("\n" + "="*70)
        print("RUNNING MAIN DEMO WITH WORKING MODEL")
        print("="*70)

        demo_structured_agent()

        # Test advanced agent
        print("\n" + "="*60)
        print("ADVANCED AGENT WITH TOOLS")
        print("="*60)

        advanced_agent = AdvancedAgent(model=working_models[0])

        # Test calculator tool
        print("\n🔧 Testing Calculator Tool:")
        tool_response1 = advanced_agent.process_with_tools("Calculate 25 * 4 + 10")
        print("Tool-based response:")
        print(json.dumps(tool_response1, indent=2))

        # Test time tool
        print("\n⏰ Testing Time Tool:")
        tool_response2 = advanced_agent.process_with_tools("What's the current time?")
        print("Tool-based response:")
        print(json.dumps(tool_response2, indent=2))

    else:
        print("\n❌ No working models found. Please check:")
        print("1. Your Groq API key is valid")
        print("2. You have internet access")
        print("3. Check Groq documentation for current models")

# Simple chat interface for testing
def quick_chat():
    """Quick chat interface for testing"""
    agent = SimpleAgent(model="llama-3.1-8b-instant")

    print("\n💬 Quick Chat Interface (type 'quit' to exit)")
    print("Available models:", agent.get_available_models())

    while True:
        user_input = input("\nYou: ")
        if user_input.lower() in ['quit', 'exit', 'bye']:
            break

        response = agent.get_structured_response(user_input)
        print("\nAssistant:")
        print(json.dumps(response, indent=2))

# Uncomment to run quick chat
# quick_chat()

ERROR: Operation cancelled by user
🚀 Starting Groq Agent with Current Models...
🧪 Testing ALL Potential Groq Models...

Testing: llama-3.1-8b-instant
✅ SUCCESS - llama-3.1-8b-instant
   Response: Solar energy has several benefits....

Testing: llama-3.2-1b-preview
❌ FAILED - llama-3.2-1b-preview
   Reason: Model decommissioned

Testing: llama-3.2-3b-preview
❌ FAILED - llama-3.2-3b-preview
   Reason: Model decommissioned

Testing: llama-3.2-90b-vision-preview
❌ FAILED - llama-3.2-90b-vision-preview
   Reason: Model decommissioned

Testing: llama-3.2-11b-vision-preview
❌ FAILED - llama-3.2-11b-vision-preview
   Reason: Model decommissioned

Testing: llama-3.2-90b-text-preview
❌ FAILED - llama-3.2-90b-text-preview
   Reason: Model decommissioned

Testing: llama-3.2-1b-text-preview
❌ FAILED - llama-3.2-1b-text-preview
   Reason: Failed to get response: Error code: 404 - {'error': {'message': 'The model `llama-3.2-1b-text-previe...

Testing: llama-3.2-3b-text-preview
❌ FAILED - llama-3.2-3b